In [1]:
import pandas as pd
import numpy as np

import hopsworks

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    fix_datatypes,
    add_date_features,
    remove_playoff_games,
    add_rolling_home_visitor,
    process_games_consecutively,
    add_matchups,
    add_past_performance_all,
    combine_new_features,
    process_x_minus_y,  
    remove_non_rolling,
)

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')



### Get Data

In [2]:
games = pd.read_csv(DATAPATH / "games.csv")

### Data Processing

In [3]:
games = process_games(games) 
games = add_TARGET(games)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Feature Engineering

In [4]:
def process_features(df):
    
    home_visitor_roll_list = [3, 7, 10]
    all_roll_list = [3, 7, 10, 15]
    
    df = fix_datatypes(df)
    df = add_date_features(df)
    df = remove_playoff_games(df)
    df = add_rolling_home_visitor(df, "HOME", home_visitor_roll_list)
    df = add_rolling_home_visitor(df, "VISITOR", home_visitor_roll_list)
    
    df_consecutive = process_games_consecutively(df)
    df_consecutive = add_matchups(df_consecutive, home_visitor_roll_list)
    df_consecutive = add_past_performance_all(df_consecutive, all_roll_list)

    #add these features back to main dataframe
    df = combine_new_features(df,df_consecutive) 
    
    df = remove_non_rolling(df)
    
    df = process_x_minus_y(df)
    
    return df

games = process_features(games)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

### Backfill Feature Store

In [5]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


TypeError: __init__() got an unexpected keyword argument 'creation_status'

**Rolling Stats Features**

In [ ]:
rolling_stats_fg = fs.get_or_create_feature_group(
    name="rolling_stats",
    version=1,
    description="Rolling averages and current win/lose streaks",
    primary_key=["GAME_ID"],
)

In [ ]:
rolling_stats_fg.insert(games, write_options={"wait_for_job" : False})

In [6]:
project.close()

NameError: name 'project' is not defined